In [13]:
from pathlib import Path

def find_xlsx_files(directory):
    path = Path(directory)
    return [str(file) for file in path.glob("*.xlsx") if not file.name.startswith('~$')]  # Exclude files starting with ~$ and no recursive search

# Example usage:
directory = "../../data/Forms"
xlsx_files = find_xlsx_files(directory)
for file in xlsx_files:
    print(file)

../../data/Forms/employeeAvailabilityForm_Template.xlsx


In [14]:
import openpyxl
from datetime import datetime
from pprint import pprint

def extract_ta_availability(file_path: str):
    # Load the workbook and select the sheet
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook['TA_Availability']
    
    # Extracting TA details from the top of the sheet
    ta_name = sheet['B2'].value
    mac_id = sheet['B3'].value
    contract_hours = sheet['B4'].value
    ta_type = sheet['B5'].value
    
    # Extracting the time slots and availability for each day
    avialability = []
    
    for row in sheet.iter_rows(min_row=7, min_col=1, max_col=7, values_only=True):
        start_time_str, end_time_str, monday, tuesday, wednesday, thursday, friday = row
        
        # Converting the start and end times to time objects (ignoring the date)
        if isinstance(start_time_str, datetime):
            start_time = start_time_str.time()  # If it's already a datetime, extract time
        elif start_time_str:
            start_time = datetime.strptime(start_time_str, "%H:%M").time()
        else:
            start_time = None
        
        if isinstance(end_time_str, datetime):
            end_time = end_time_str.time()  # If it's already a datetime, extract time
        elif end_time_str:
            end_time = datetime.strptime(end_time_str, "%I:%M %p").time()  # Assuming the format is 6:30 PM
        else:
            end_time = None
        
        avialability.append({
            "start_time": start_time,
            "end_time": end_time,
            "monday": monday,
            "tuesday": tuesday,
            "wednesday": wednesday,
            "thursday": thursday,
            "friday": friday
        })
    
    return {
        "name": ta_name,
        "mac_id": mac_id,
        "contract_hours": contract_hours,
        "type": ta_type,
        "avialability": avialability
    }

# Test the function with the first file in the list
extracted = extract_ta_availability(xlsx_files[0])
pprint(extracted.get("avialability"))


[{'end_time': datetime.time(17, 30),
  'friday': 'Unavailable',
  'monday': 'Undesired',
  'start_time': datetime.time(14, 30),
  'thursday': 'Desired',
  'tuesday': 'Undesired',
  'wednesday': 'Unavailable'},
 {'end_time': datetime.time(21, 30),
  'friday': 'Unavailable',
  'monday': 'Desired',
  'start_time': datetime.time(18, 30),
  'thursday': 'Desired',
  'tuesday': 'Desired',
  'wednesday': 'Desired'}]
